# SQL Server 2017 Python integration

We will look at how to build an ML model and then deploy it to SQL server. 

This this example we will use Pandas and SKLearn. 

Our attempt is to predict the rental amount per month. We will do this using regression analysis. 

<b style="color: red"> Terry! Have you done the intro session yet? (In SQL Server).  </b>

In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from revoscalepy import RxComputeContext, RxInSqlServer, RxSqlServerData
from revoscalepy import rx_import

Make a connection to SQL Server. 

Define which columns we want. This bit is important. 
SKLearn if you recall needs to work with categorical data. Numerical data. 

In [41]:
conn_str = 'Driver=SQL Server;Server=.;Database=TutorialDB;Trusted_Connection=True;'

#Define the columns we wish to import
column_info = {
         "Year" : { "type" : "integer" },
         "Month" : { "type" : "integer" },
         "Day" : { "type" : "integer" },
         "RentalCount" : { "type" : "integer" },
         "WeekDay" : {
             "type" : "factor",
             "levels" : ["1", "2", "3", "4", "5", "6", "7"]
         },
         "Holiday" : {
             "type" : "factor",
             "levels" : ["1", "0"]
         },
         "Snow" : {
             "type" : "factor",
             "levels" : ["1", "0"]
         }
     }



Get the data from SQL Server Table

In [28]:
data_source = RxSqlServerData(table="dbo.rental_data",
                               connection_string=conn_str, column_info=column_info)
computeContext = RxInSqlServer(
     connection_string = conn_str,
     num_tasks = 1,
     auto_cleanup = False
)

In [29]:
RxInSqlServer(connection_string=conn_str, num_tasks=1, auto_cleanup=False)

Import data source and convert to pandas dataframe

In [30]:
df = pd.DataFrame(rx_import(input_data = data_source))
print("Data frame:", df)

Rows Read: 453, Total Rows Processed: 453, Total Chunk Time: 0.016 seconds 
Data frame:      Year  Month  Day  RentalCount WeekDay Holiday Snow FHoliday FSnow  \
0    2014      1   20          445       2       1    0        1     0   
1    2014      2   13           40       5       0    0        0     0   
2    2013      3   10          456       1       0    0        0     0   
3    2014      3   31           38       2       0    0        0     0   
4    2014      4   24           23       5       0    0        0     0   
5    2015      2   11           42       4       0    0        0     0   
6    2013      4   28          310       1       0    0        0     0   
7    2014      3    8          240       7       0    0        0     0   
8    2013      4    5           22       6       0    0        0     0   
9    2015      3   29          360       1       0    0        0     0   
10   2015      4   22           20       4       0    0        0     0   
11   2014      4    1   

Get all the columns from the dataframe.

In [31]:
columns = df.columns.tolist()

Filter the columns to remove ones we don't want to use in the training

In [32]:
columns = [c for c in columns if c not in ["Year"]]

Store the variable we'll be predicting on.

In [33]:
target = "RentalCount"

Generate the training set.  Set random_state to be able to replicate results.

In [34]:
train = df.sample(frac=0.8, random_state=1)
test = df.loc[~df.index.isin(train.index)]

Lets take a look at the the shape of the training and test data.

In [35]:
print("Training set shape:", train.shape)
print("Testing set shape:", test.shape)


Training set shape: (362, 10)
Testing set shape: (91, 10)


Initialize the model class.

In [36]:
lin_model = LinearRegression()

Fit the model to the training data.

In [37]:
lin_model.fit(train[columns], train[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [23]:
# Generate our predictions for the test set.



Predictions: [  40.   38.  240.   39.  514.   48.  297.   25.  507.   24.   30.   54.
   40.   26.   30.   34.   42.  390.  336.   37.   22.   35.   55.  350.
  252.  370.  499.   48.   37.  494.   46.   25.  312.  390.   35.   35.
  421.   39.  176.   21.   33.  452.   34.   28.   37.  260.   49.  577.
  312.   24.   24.  390.   34.   64.   26.   32.   33.  358.  348.   25.
   35.   48.   39.   44.   58.   24.  350.  651.   38.  468.   26.   42.
  310.  709.  155.   26.  648.  617.   26.  846.  729.   44.  432.   25.
   39.   28.  325.   46.   36.   50.   63.]


In [38]:
lin_predictions = lin_model.predict(test[columns])
print("Predictions:", lin_predictions)

Predictions: [  40.   38.  240.   39.  514.   48.  297.   25.  507.   24.   30.   54.
   40.   26.   30.   34.   42.  390.  336.   37.   22.   35.   55.  350.
  252.  370.  499.   48.   37.  494.   46.   25.  312.  390.   35.   35.
  421.   39.  176.   21.   33.  452.   34.   28.   37.  260.   49.  577.
  312.   24.   24.  390.   34.   64.   26.   32.   33.  358.  348.   25.
   35.   48.   39.   44.   58.   24.  350.  651.   38.  468.   26.   42.
  310.  709.  155.   26.  648.  617.   26.  846.  729.   44.  432.   25.
   39.   28.  325.   46.   36.   50.   63.]


Compute error between our test predictions and the actual values.

Here we are looking at the mean-squared-error MSE. This is the difference from the predicted line to the actual values, then take the mean. 

In [39]:
lin_mse = mean_squared_error(lin_predictions, test[target])
print("Computed error:", lin_mse)

Computed error: 8.94148696545e-27


So that was a model trained using SQL Server, however we now need to deploy it somewhere. 